In [ ]:
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("FRED_API_KEY")
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"


ENERGY_SERIES = {
    # Green Energy
    "Renewable_Electricity_Generation": "ELECGENRENEWUS",
    "Solar_Generation": "ELECGENSOLUS",
    "Wind_Generation": "ELECGENWNDUS",
    "Hydropower_Generation": "ELECGENHYCUS",

    # Natural Gas
    "Natural_Gas_Prices": "NGAC1",
    "Natural_Gas_Exports": "NGEXUS",
    "Natural_Gas_Imports": "NGIMUS",
    "Natural_Gas_Consumption": "NGCONS",

    # Petroleum & Crude Oil
    "Crude_Oil_Rig_Count": "MCRRIPUS",
    "Petroleum_Imports": "PETIMPUS",
    "Petroleum_Exports": "PETEXPUS",
    "Crude_Oil_Reserves": "EOICRUUS",
    "WTI_Crude_Oil_Price": "DCOILWTICO",

    # Overall Energy Sector
    "Electric_Power_Prod_Trans_Dist": "IPG2211A2N",
    "Natural_Gas_Distribution": "IPG2212A2N",
    "Oil_and_Gas_Extraction": "IPG2111A2N"
}

def fetch_fred_series(series_id, start_date="2000-01-01"):
    params = {
        "series_id": series_id,
        "api_key": API_KEY,
        "file_type": "json",
        "observation_start": start_date
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch {series_id}: {response.status_code}")
        return pd.DataFrame()

    data = response.json()
    if "observations" not in data:
        print(f"No observations found for {series_id}")
        return pd.DataFrame()

    df = pd.DataFrame(data["observations"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["date"] = pd.to_datetime(df["date"])
    df = df[["date", "value"]].set_index("date")
    df.columns = [series_id]
    return df

#fetch
all_energy_series = []

for label, series_id in ENERGY_SERIES.items():
    print(f"Fetching {label} ({series_id})")
    df = fetch_fred_series(series_id)
    if not df.empty:
        df.columns = [label]
        all_energy_series.append(df)

#combine
energy_df = pd.concat(all_energy_series, axis=1)

#save .csv
energy_df.to_csv("fred_energy_data.csv")
print(energy_df.tail())

Fetching Renewable_Electricity_Generation (ELECGENRENEWUS)
Failed to fetch ELECGENRENEWUS: 400
Fetching Solar_Generation (ELECGENSOLUS)
Failed to fetch ELECGENSOLUS: 400
Fetching Wind_Generation (ELECGENWNDUS)
Failed to fetch ELECGENWNDUS: 400
Fetching Hydropower_Generation (ELECGENHYCUS)
Failed to fetch ELECGENHYCUS: 400
Fetching Natural_Gas_Prices (NGAC1)
Failed to fetch NGAC1: 400
Fetching Natural_Gas_Exports (NGEXUS)
Failed to fetch NGEXUS: 400
Fetching Natural_Gas_Imports (NGIMUS)
Failed to fetch NGIMUS: 400
Fetching Natural_Gas_Consumption (NGCONS)
Failed to fetch NGCONS: 400
Fetching Crude_Oil_Rig_Count (MCRRIPUS)
Failed to fetch MCRRIPUS: 400
Fetching Petroleum_Imports (PETIMPUS)
Failed to fetch PETIMPUS: 400
Fetching Petroleum_Exports (PETEXPUS)
Failed to fetch PETEXPUS: 400
Fetching Crude_Oil_Reserves (EOICRUUS)
Failed to fetch EOICRUUS: 400
Fetching WTI_Crude_Oil_Price (DCOILWTICO)
Fetching Electric_Power_Prod_Trans_Dist (IPG2211A2N)
Fetching Natural_Gas_Distribution (IPG221